# Exercise: Planning events for visitors

<font color='blue'><b>Goal:</b></font> Solve an application problem using linear programs, although a priori, integer variables seem necessary.

<font color='blue'><b>Python packages required:</b></font> `pulp`

## Problem setting

Imagine that you are the director of a big zoo. Every day, several groups come to visit, and these groups can enjoy a special offer: They can participate in events organized for them, as for example animal feedings, guided tours behind the scenes, film screenings, etc. So far, there was one event offered every full hour from 8:00 in the morning until 18:00 in the evening, and groups could freely choose where to go.

Each of the events incurs fixed positive costs that are independent of the actual number of participating groups. Moreover, you know in advance at what time groups plan to come to your zoo and leave again, i.e., you know for each group in which consecutive events they could potentially participate.

In an effort to save costs, your goal is to only offer a subset of the current events based on a daily decision that you make after knowing the groups' schedules. More concretely, you want to offer a minimum cost subset of the current events such that every group has the chance to participate in at least one event.

### An concrete example

As an example, the event costs could be the ones given by the following table.

| Time       | 8:00 | 9:00 | 10:00 | 11:00 | 12:00 | 13:00 | 14:00 | 15:00 | 16:00 | 17:00 | 18:00 |
|------------|:----:|:----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|
| Event cost |  $3$ |  $5$ |  $7$  |  $9$  |  $6$  |  $3$  |  $8$  |  $5$  |  $7$  |  $6$  |  $4$  |

Moreover, the groups might have announced the following schedules, where an `x` indicates that the group will be in the zoo at the indicated time.

| Time    | 8:00 | 9:00 | 10:00 | 11:00 | 12:00 | 13:00 | 14:00 | 15:00 | 16:00 | 17:00 | 18:00 |
|---------|:----:|:----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|
| Group 1 |   .  |   x  |   x   |   x   |   x   |   .   |   .   |   .   |   .   |   .   |   .   |
| Group 2 |   .  |   .  |   .   |   .   |   x   |   x   |   x   |   .   |   .   |   .   |   .   |
| Group 3 |   .  |   .  |   .   |   x   |   x   |   x   |   x   |   x   |   .   |   .   |   .   |
| Group 4 |   x  |   x  |   x   |   x   |   .   |   .   |   .   |   .   |   .   |   .   |   .   |
| Group 5 |   .  |   .  |   .   |   .   |   .   |   .   |   x   |   x   |   x   |   x   |   x   |
| Group 6 |   .  |   .  |   .   |   x   |   x   |   x   |   x   |   x   |   .   |   .   |   .   |
| Group 7 |   .  |   .  |   .   |   .   |   .   |   .   |   x   |   x   |   x   |   x   |   .   |
| Group 8 |   .  |   .  |   x   |   x   |   x   |   x   |   x   |   x   |   x   |   .   |   .   |

Thus, the event at 16:00 costs $7$ units, and if you decide to offer this event, groups 5, 7, and 8 can participate. A possible set of events you could offer such that at every group has the chance to participate in at least one of them consists of the events at 11:00 (groups 1, 3, 4, 6, and 8 can participate) and 14:00 (groups 2, 3, 5, 6, 7, and 8 can participate). But note that the total cost will then be $17$, which is more than if you decide to offer the events at 10:00, 13:00, and 15:00 (the latter set has total cost $15$, and it is easy to see that every group can participate in at least one of the chosen events).



## Modelling and solving the problem as a linear program
When modelling the problem as an optimization problem, a natural idea is to introduce variables $x_e$ for each event $e$, where $x_e=1$ indicates that event $e$ is offered, and $x_e=0$ indicates that event $e$ is not offered. Like this, every choice of events corresponds to a $0$-$1$ vector with one entry for each event.

<font color="blue"><b>Your first task:</b></font> Write a linear program such that vertex solutions of your linear program correspond to choices of events in the sense specified above. Prove that this is indeed the case! *(This is not a coding exercise.)*

*...write your answer here...*

<font color="blue"><b>Your second task:</b></font> Implement the linear program you came up with in the first part, and use it to find an optimal solution of the given problem! To this end, you can use the cost vector and schedule matrix given below (note that the schedule matrix is a $0$-$1$ matrix that has one column per event and one row per group, and the table entry in row $i$ and column $j$ equals $1$ iff Group $i$ is in the zoo at the time event $j$ takes place).

In [ ]:
costs = [3, 5, 7, 9, 6, 3, 8, 5, 7, 6, 4]
schedules = [
    [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
    [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0]
]

*Remark:* Note that the above approach requires a linear programming solver that returns vertex solutions. If you use an LP solving package, it is often not clear what methods are used in the background. The Simplex Method would, for example, guarantee vertex solutions, while others, as for example interior point methods, don't. There are various ways to find optimal vertex solutions, two of them are the following:
- Perturbing the objective function: Optimal non-vertex solutions can only exist if the optimal solution is not unique. Small perturbations of the objective function can help to make solutions unique. The unique solution must then be a vertex, hence every exact solver will return the vertex solution. Note that perturbations have to be small enough so that the objective function value does not change!
- Modifying a non-vertex solution: From a non-vertex solution, we can identify tight constraints, and hence the linear subspace defining the minimal face of the feasible region in which the optimal solution is contained. Moving in a non-zero direction inside the subspace until we hit new constraints, we can iteratively increase the number of tight constraints until a full-rank system and hence a vertex solution is reached.

In [ ]:
## Your code goes here.


<font color="blue"><b>Bonus task:</b></font> Imagine that the costs of all events are the same, i.e., $c\equiv 1$. This objective is your choice if you simply want to minimize the number of events. Observe, for example, that in the concrete setting given above, two events would be enough: Offering the events at 11:00 and 14:00, every group can attend at least one of the events.

Can you come up with an efficient combinatorial algorithm for selecting a minimum-cardinality subset of events such that every group can attend at least one event? What is the running time of your algorithm? For these questions, assume that the input is a cost vector $c>0$ and an $m\times n$ schedule matrix with entries in $\{0,1\}$ as seen above that shows the availability intervals of $m$ groups for $n$ events.